# Setup and Imports

In [ ]:
# General imports
import numpy as np
import os

# TensorFlow/Keras imports
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# For deployment
from flask import Flask, request, jsonify
import tensorflow as tf

# Model Loading and Tuning

In [ ]:
# Load the pre-trained model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(14, activation='softmax')(x)  # Adjust the number of classes

# Compile the final model
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Paths to data directories
train_dir = 'data/train'
val_dir = 'data/val'

# Data generators
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1.0/255)

# Data loaders
train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=(224, 224), batch_size=32, class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir, target_size=(224, 224), batch_size=32, class_mode='categorical'
)

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,  # Adjust as needed
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=val_generator.samples // val_generator.batch_size
)

# Save the model for later use
model.save('facial_recognition_model.h5')

# Inference

In [ ]:
# Load the saved model
model = load_model('facial_recognition_model.h5')

# Function to preprocess images
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Function for prediction
def predict(image_path):
    processed_image = preprocess_image(image_path)
    predictions = model.predict(processed_image)
    class_index = np.argmax(predictions)
    confidence = np.max(predictions)
    return class_index, confidence

# Test the inference function
image_path = 'test_image.jpg'  # Replace with your test image path
class_index, confidence = predict(image_path)
print(f"Predicted Class: {class_index}, Confidence: {confidence:.2f}")

# Flask Deployment

In [ ]:
# Flask API for deployment
app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict_api():
    file = request.files['image']
    if not file:
        return jsonify({'error': 'No file uploaded'}), 400

    # Save and preprocess the image
    image_path = './temp.jpg'
    file.save(image_path)
    class_index, confidence = predict(image_path)

    return jsonify({'class_index': int(class_index), 'confidence': float(confidence)})

# Run the Flask app (execute only when running locally)
if __name__ == '__main__':
    app.run(debug=True)

# Testing API

In [ ]:
import requests

url = 'http://127.0.0.1:5000/predict'
files = {'image': open('test_image.jpg', 'rb')}
response = requests.post(url, files=files)

print(response.json())